![hackio](https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_celeste@4x.png?raw=true)


# Laboratorio A/B Testing No Paramétricos

**Aerolínea:** 
Alas del Mundo

**Descripción de la Aerolínea**

Alas del Mundo es una aerolínea internacional que opera una amplia red de rutas a nivel mundial. La compañía se enorgullece de ofrecer vuelos cómodos y asequibles a una variedad de destinos, atrayendo a una diversa gama de clientes.

**Objetivo del Análisis**

El objetivo de este ejercicio es determinar si existe una diferencia significativa en el número de vuelos reservados entre pasajeros con diferentes niveles educativos y si existe una diferencia significativa entre el género de sus clientes y su salario. La aerolínea desea entender mejor a sus clientes para poder personalizar sus campañas de marketing y mejorar sus estrategias de ventas.

**Set de datos**

El conjunto de datos *alas_del_mundo.csv* contiene información sobre clientes de la aerolínea Alas del Mundo. A continuación, tenemos una descripción de lo que nos podemos encontrar en el conjunto de datos:

- **Loyalty Number**: Número de registro de cada uno de los clientes.

- **Province**: Provincia de residencia del cliente.

- **City**: Ciudad de residencia del cliente.

- **Gender**: Género con el que se identifica el cliente.

- **Education**: Nivel educativo del cliente.

- **Salary**: Salario anual del cliente.

- **Loyalty Card**: Nivel que tiene el cliente dentro del programa de fidelización de la aerolínea.

- **CLV**: Valor total del cliente para la aerolínea a lo largo de toda su relación.

- **Enrollment Type**: Forma de registro del cliente a la aerolínea.

- **Enrollment Year**: Año de inscripción.

- **Enrollment Month**: Mes de inscripción. 

- **Year**: Año en el que se realizan los vuelos.

- **Month**: Mes en el que se realizan los vuelos.

- **Flights Booked**: Vuelos reservados por el cliente en un mes y año concretos.

- **Distance**: Distancia de vuelo del cliente en un mes y año concreto

- **Points Accumulated**: Puntos acumulados por el cliente en un mes y año concretos.

## Ejercicio 1:

Antes de realizar el experimento A/B y formular hipótesis, es fundamental comprender a fondo el conjunto de datos y sus características. Para lograr esto, debes llevar a cabo un análisis exploratorio detallado de los datos (EDA), con el fin de familiarizarte con ellos y entender la información disponible.



In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import sys

sys.path.append('..') #pongo los 2 puntos para que salga de la carpeta
import src.support as s

In [2]:
df = pd.read_csv('C:\\Users\\DELL\\Git\\LAB_AB_no_parametrico\\datos\\alas_del_mundo.csv')

In [3]:
df.head()

,Loyalty Number,Province,City,Gender,Education,Salary,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Year,Month,Flights Booked,Distance,Points Accumulated
0,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,1,0,0,0.0
1,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,2,3,2823,282.0
2,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,3,0,0,0.0
3,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,4,0,0,0.0
4,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2018,10,6,3352,335.0


In [4]:
nuevas_columnas = {columna: columna.strip().lower().replace(' ', '_') for columna in df.columns}
df.rename(columns=nuevas_columnas, inplace=True)

In [5]:
df.head()

,loyalty_number,province,city,gender,education,salary,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,year,month,flights_booked,distance,points_accumulated
0,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,1,0,0,0.0
1,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,2,3,2823,282.0
2,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,3,0,0,0.0
3,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2017,4,0,0,0.0
4,480934,Ontario,Toronto,Female,Bachelor,83236.0,Star,3839.14,Standard,2016,2,2018,10,6,3352,335.0


## Primera exploración
- Tenemos 405624 filas, distribuidas en 16 columnas.
- Tenemos 1864 duplicados, que me he cargado.
- Tenemos valores negativos en la columna de salarios, voy a convertirlos a valor absoluto para que sean unos datos que me sumen a la media.
- Tengo toda una columna de de grupo educativo que no se conoce su educación (College). Al no conocer estos datos, pasamos por alto su trabajo ya que no podemos sacar dicha información de otra parte.
- Contaba con valores negativos en mis salarios, por lo que decidí convertir a positivos
- Después de convertir los negativos a positivos, mi media salarial es de 79371 anualmente.
- Cabe destacar que mis datos se reparten entre 2017-2018
- La mayor parte de vuelos lo hacen las mujeres.
- La residencia de los clientes de la aerolínea, son mayormente residentes en Toronto, Ontario.

In [36]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
loyalty_number,403760.0,549875.383713,258961.514684,100018.00,326699.00,550598.00,772152.00,999986.00
salary,301500.0,79371.732902,34726.433958,9081.00,59262.00,73479.00,88606.00,407228.00
clv,403760.0,7990.864857,6863.317160,1898.01,3981.78,5776.34,8937.12,83325.38
enrollment_year,403760.0,2015.250716,1.979427,2012.00,2014.00,2015.00,2017.00,2018.00
enrollment_month,403760.0,6.667555,3.398829,1.00,4.00,7.00,10.00,12.00
year,403760.0,2017.500352,0.500000,2017.00,2017.00,2018.00,2018.00,2018.00
month,403760.0,6.501335,3.451982,1.00,4.00,7.00,10.00,12.00
flights_booked,403760.0,4.134050,5.230064,0.00,0.00,1.00,8.00,21.00
distance,403760.0,1214.460979,1434.098521,0.00,0.00,525.00,2342.00,6293.00
points_accumulated,403760.0,124.263761,146.696179,0.00,0.00,53.00,240.00,676.50


In [37]:
df.describe().median().T

loyalty_number        476817.691857
salary                 76425.366451
clv                     7427.091008
enrollment_year         2015.125358
enrollment_month           6.833777
year                    2017.750176
month                      6.750667
flights_booked             4.682057
distance                1324.279750
points_accumulated       135.479970
dtype: float64

In [7]:
df.select_dtypes('O').describe()

,province,city,gender,education,loyalty_card,enrollment_type
count,405624,405624,405624,405624,405624,405624
unique,11,29,2,5,3,2
top,Ontario,Toronto,Female,Bachelor,Star,Standard
freq,130896,81096,203640,253752,184224,382200


In [13]:
df[df.duplicated()]

,loyalty_number,province,city,gender,education,salary,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,year,month,flights_booked,distance,points_accumulated
121,193662,Yukon,Whitehorse,Male,Bachelor,51124.0,Star,3844.57,Standard,2012,5,2017,1,0,0,0.0
160,193662,Yukon,Whitehorse,Male,Bachelor,51124.0,Star,3844.57,Standard,2012,5,2018,8,0,0,0.0
165,193662,Yukon,Whitehorse,Male,Bachelor,51124.0,Star,3844.57,Standard,2012,5,2018,11,0,0,0.0
2977,746226,British Columbia,Whistler,Female,Bachelor,63501.0,Star,4089.04,Standard,2018,1,2017,1,0,0,0.0
2979,746226,British Columbia,Whistler,Female,Bachelor,63501.0,Star,4089.04,Standard,2018,1,2017,2,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381799,373638,Ontario,Thunder Bay,Female,Bachelor,92501.0,Star,11073.11,Standard,2014,10,2018,4,0,0,0.0
381801,373638,Ontario,Thunder Bay,Female,Bachelor,92501.0,Star,11073.11,Standard,2014,10,2018,5,0,0,0.0
381803,373638,Ontario,Thunder Bay,Female,Bachelor,92501.0,Star,11073.11,Standard,2014,10,2018,6,0,0,0.0
381805,373638,Ontario,Thunder Bay,Female,Bachelor,92501.0,Star,11073.11,Standard,2014,10,2018,7,0,0,0.0


In [15]:
df = df.drop_duplicates()

In [16]:
df.shape

(403760, 16)

In [25]:
df.isnull().sum()

loyalty_number             0
province                   0
city                       0
gender                     0
education                  0
salary                102260
loyalty_card               0
clv                        0
enrollment_type            0
enrollment_year            0
enrollment_month           0
year                       0
month                      0
flights_booked             0
distance                   0
points_accumulated         0
dtype: int64

In [26]:
dfprueba = df[df['salary'].isna()]

In [28]:
dfprueba['education'].unique()

array(['College'], dtype=object)

## Ejercicio 2:
Determina si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

 Pasos a seguir:

   - Definición de la hipótesis.
   - Prueba de hipótesis
   - Análisis de resultados
   - Conclusiones
   


### H0: no existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos
### H1: existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos

#### Segunda exploración: vuelos y educación
- En cuanto a educación, tenemos 5 valores únicos.

- En cuanto a la cantidad de vuelos, el mínimo es 0 y el máximo es 21 vuelos. Su media es de 4.13 medias y su mediana es de 4.68. Por lo que su distribución es normal.

- El grupo de educación que tiene un mayor conteo de vuelos es el de Bachelor, dejando en último lugar al grupo de Máster. A la vez, el grupo educativo con mayor frecuencia es el de Bachelor.


In [19]:
df['education'].unique()

array(['Bachelor', 'College', 'Master', 'High School or Below', 'Doctor'],
      dtype=object)

In [21]:
df.groupby('education')['flights_booked'].count()

education
Bachelor                252567
College                 102260
Doctor                   17731
High School or Below     18915
Master                   12287
Name: flights_booked, dtype: int64

In [33]:
df[df['salary'] < 0].head()

,loyalty_number,province,city,gender,education,salary,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,year,month,flights_booked,distance,points_accumulated
26328,542976,Quebec,Montreal,Male,High School or Below,-49830.0,Star,24127.5,2018 Promotion,2018,3,2017,1,0,0,0.0
26329,542976,Quebec,Montreal,Male,High School or Below,-49830.0,Star,24127.5,2018 Promotion,2018,3,2017,2,0,0,0.0
26330,542976,Quebec,Montreal,Male,High School or Below,-49830.0,Star,24127.5,2018 Promotion,2018,3,2017,3,0,0,0.0
26331,542976,Quebec,Montreal,Male,High School or Below,-49830.0,Star,24127.5,2018 Promotion,2018,3,2017,4,0,0,0.0
26332,542976,Quebec,Montreal,Male,High School or Below,-49830.0,Star,24127.5,2018 Promotion,2018,3,2017,5,0,0,0.0


In [ ]:
df['salary'] =  df['salary'].apply(lambda x: abs(x) if x < 0 else x) # tenemos aqui la forma de reconvertir los alarios negativos a positivos

In [35]:
df[df['salary'] < 0].head()

,loyalty_number,province,city,gender,education,salary,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,year,month,flights_booked,distance,points_accumulated


In [11]:
s.exploracion_dataframe(df, 'education')

El número de datos es 405624 y el de columnas es 16

 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 1864

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


,%_nulos
salary,25.312112



 ..................... 

Los tipos de las columnas son:


,tipo_dato
loyalty_number,int64
province,object
city,object
gender,object
education,object
salary,float64
loyalty_card,object
clv,float64
enrollment_type,object
enrollment_year,int64



 ..................... 

Los valores que tenemos para las columnas categóricas son: 
La columna province tiene las siguientes valore únicos:


...

,count,mean,std,min,25%,50%,75%,max
loyalty_number,19008.0,558216.804293,258662.751947,101148.00,332180.75,561366.00,782612.75,994541.0
salary,19008.0,60986.544192,12723.916043,-49830.00,52044.75,61806.00,71344.00,93875.0
clv,19008.0,7722.940328,6657.946709,2272.34,3707.28,5269.28,8991.98,43290.5
enrollment_year,19008.0,2015.193182,1.984666,2012.00,2013.00,2015.00,2017.00,2018.0
enrollment_month,19008.0,6.609848,3.457341,1.00,4.00,7.00,10.00,12.0
year,19008.0,2017.500000,0.500013,2017.00,2017.00,2017.50,2018.00,2018.0
month,19008.0,6.500000,3.452143,1.00,3.75,6.50,9.25,12.0
flights_booked,19008.0,4.155777,5.234551,0.00,0.00,1.00,8.00,21.0
distance,19008.0,1225.585543,1435.875594,0.00,0.00,580.00,2363.25,6032.0
points_accumulated,19008.0,125.176770,146.493283,0.00,0.00,59.50,242.00,661.5



 ..................... 

Los principales estadísticos de las columnas categóricas para el Doctor son: 


,count,unique,top,freq
province,17856,11,Ontario,5640
city,17856,29,Toronto,3312
gender,17856,2,Female,9024
education,17856,1,Doctor,17856
loyalty_card,17856,3,Star,8448
enrollment_type,17856,2,Standard,17088



 ..................... 

Los principales estadísticos de las columnas numéricas para el Doctor son: 


,count,mean,std,min,25%,50%,75%,max
loyalty_number,17856.0,570729.143817,253980.381625,101594.00,359104.5000,581403.50,785276.250,999911.00
salary,17856.0,178335.439516,70307.642502,48109.00,124343.7500,180440.00,235069.250,407228.00
clv,17856.0,7815.538978,6466.157567,2119.89,4056.4125,5619.19,8580.735,58207.13
enrollment_year,17856.0,2015.072581,1.949028,2012.00,2013.0000,2015.00,2017.000,2018.00
enrollment_month,17856.0,6.896505,3.389085,1.00,4.0000,7.00,10.000,12.00
year,17856.0,2017.500000,0.500014,2017.00,2017.0000,2017.50,2018.000,2018.00
month,17856.0,6.500000,3.452149,1.00,3.7500,6.50,9.250,12.00
flights_booked,17856.0,4.146281,5.250093,0.00,0.0000,1.00,8.000,21.00
distance,17856.0,1223.176131,1443.825996,0.00,0.0000,534.50,2368.000,6293.00
points_accumulated,17856.0,125.057556,147.555733,0.00,0.0000,54.00,241.000,676.50


## Ejercicio 3:
Determina si existe una diferencia significativa en el salario de los clientes en función del género de los mismos.

Pasos a seguir:
   - Definición de la hipótesis.
   - Prueba de hipótesis
   - Análisis de resultados
   - Conclusiones

H0: no existe una diferencia significativa en el salario de los clientes en función del género de los mismos
H1: existe una diferencia significativa en el salario de los clientes en función del género de los mismos

In [12]:
s.exploracion_dataframe(df, 'gender')

El número de datos es 405624 y el de columnas es 16

 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 1864

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


,%_nulos
salary,25.312112



 ..................... 

Los tipos de las columnas son:


,tipo_dato
loyalty_number,int64
province,object
city,object
gender,object
education,object
salary,float64
loyalty_card,object
clv,float64
enrollment_type,object
enrollment_year,int64



 ..................... 

Los valores que tenemos para las columnas categóricas son: 
La columna province tiene las siguientes valore únicos:


...

,count,mean,std,min,25%,50%,75%,max
loyalty_number,203640.0,548039.567590,259443.515793,100018.00,324364.00,547500.00,773593.00,999986.00
salary,153072.0,79260.219034,34384.857094,-58486.00,59265.00,73663.50,88978.00,407228.00
clv,203640.0,7999.522370,6888.419380,1898.01,3980.84,5757.44,8982.85,83325.38
enrollment_year,203640.0,2015.276841,1.972972,2012.00,2014.00,2015.00,2017.00,2018.00
enrollment_month,203640.0,6.661049,3.401924,1.00,4.00,7.00,10.00,12.00
year,203640.0,2017.500000,0.500001,2017.00,2017.00,2017.50,2018.00,2018.00
month,203640.0,6.500000,3.452061,1.00,3.75,6.50,9.25,12.00
flights_booked,203640.0,4.090542,5.214236,0.00,0.00,1.00,8.00,21.00
distance,203640.0,1204.586987,1434.339525,0.00,0.00,447.00,2328.00,6240.00
points_accumulated,203640.0,123.293699,146.788139,0.00,0.00,46.00,238.00,676.50



 ..................... 

Los principales estadísticos de las columnas categóricas para el Male son: 


,count,unique,top,freq
province,201984,11,Ontario,66456
city,201984,29,Toronto,40824
gender,201984,1,Male,201984
education,201984,5,Bachelor,125880
loyalty_card,201984,3,Star,92376
enrollment_type,201984,2,Standard,190464



 ..................... 

Los principales estadísticos de las columnas numéricas para el Male son: 


,count,mean,std,min,25%,50%,75%,max
loyalty_number,201984.0,552052.562025,258406.878328,100102.00,329004.5000,553404.00,770470.000,999982.00
salary,149880.0,79277.616173,35601.765489,-57297.00,59262.0000,73277.00,88270.000,381124.00
clv,201984.0,7984.368213,6838.623275,1898.01,4004.1275,5792.14,8901.215,83325.38
enrollment_year,201984.0,2015.228018,1.985618,2012.00,2014.0000,2015.00,2017.000,2018.00
enrollment_month,201984.0,6.675024,3.397583,1.00,4.0000,7.00,10.000,12.00
year,201984.0,2017.500000,0.500001,2017.00,2017.0000,2017.50,2018.000,2018.00
month,201984.0,6.500000,3.452061,1.00,3.7500,6.50,9.250,12.00
flights_booked,201984.0,4.139764,5.236766,0.00,0.0000,1.00,8.000,21.00
distance,201984.0,1213.208328,1431.950935,0.00,0.0000,526.00,2343.000,6293.00
points_accumulated,201984.0,124.095014,146.408997,0.00,0.0000,54.00,240.000,675.00
